In [1]:
from op_op.load import load_events, load_motion, load_raw_calcium
from op_op.alignment import demarcate_trials, demarcate_latency_from_event

import os
from pathlib import Path
from IPython.display import display

In [2]:
DATA_DIR = Path(os.environ.get("DATA_DIR")) / "raw"
MOUSE_NAME = "example-mouse"
SESSION_NAME = "rdt-d1"

## Trial alignment for calcium data


#### Raw Data

In [3]:
df_traces = load_raw_calcium(DATA_DIR, MOUSE_NAME, SESSION_NAME)
df_events = load_events(DATA_DIR, MOUSE_NAME, SESSION_NAME)

# only keep events that are not omissions
df_events = df_events[~df_events["was_omission"]]

display(df_traces.iloc[:, :10].head(3))
display(df_events.head(3))

,time,n0,n1,n2,n3,n4,n5,n6,n7,n8
0,0.0,0.001500,3.392521,-0.128361,0.871378,-0.428930,-0.048586,1.096583,0.326638,0.504377
1,0.1,-0.168614,3.386274,0.116170,-0.195632,-0.219739,-0.008950,0.797488,0.242038,0.252346
2,0.2,-0.224540,3.062015,0.124370,0.328926,-0.496898,0.012317,0.779102,0.366657,0.020969


,trial_idx,block_type,was_omission,was_forced_choice,start_time,mouse_init_time,screen_touch_time,reward_collection_time,chose_large,was_shocked
0,1,1,False,True,78.241,78.842,81.672,82.344,True,False
1,2,1,False,True,90.344,95.799,98.387,99.676,True,False
2,3,1,False,True,107.676,108.766,109.963,110.787,True,False


#### Trial Alignment

In [13]:
df1 = demarcate_trials(df_traces, df_events)
df1 = demarcate_latency_from_event(df1, df_events, event_time_col="start_time")
df1 = demarcate_latency_from_event(
    df1,
    df_events,
    event_time_col="reward_collection_time",
    t_after_event=0,
    t_before_event=10e10,
    adjust_backward=False,  # when aligning backwards
)

#### Output

In [14]:
trial1 = df1.loc[df1["trial_idx"] == 1]
display(trial1.iloc[:, :8].head(3))
display(trial1.iloc[:, :8].tail(3))

,time,latency_from_reward_collection_time,latency_from_start_time,trial_idx,n0,n1,n2,n3
783,78.3,-4.0,0.0,1.0,0.221213,0.049082,-0.264147,-0.212050
784,78.4,-3.9,0.1,1.0,-0.072269,0.183371,-0.000885,-0.508248
785,78.5,-3.8,0.2,1.0,-0.026975,-0.283777,-0.236625,-0.610011


,time,latency_from_reward_collection_time,latency_from_start_time,trial_idx,n0,n1,n2,n3
821,82.1,-0.2,3.8,1.0,-0.006157,-0.218941,-0.043499,-0.585108
822,82.2,-0.1,3.9,1.0,0.123758,-0.212039,0.047786,0.021029
823,82.3,-0.0,4.0,1.0,-0.051530,-0.027492,-0.343163,-0.104680


#### Properties of output

In [16]:
trial1_time = trial1["time"]
tmin, tmax = trial1_time.min().round(2), trial1_time.max().round(2)
print(f"Trial 1 time range: {tmin} to {tmax}")

display(
    df_events.loc[df_events["trial_idx"] == 1,][
        ["trial_idx", "start_time", "reward_collection_time"]
    ]
)

Trial 1 time range: 78.3 to 82.3


,trial_idx,start_time,reward_collection_time
0,1,78.241,82.344


## Trial alignment for motion data


#### Raw Data

In [26]:
df_motion = load_motion(DATA_DIR, MOUSE_NAME, SESSION_NAME)
df_events = load_events(DATA_DIR, MOUSE_NAME, SESSION_NAME)

df_events = df_events[~df_events["was_omission"]]


display(df_motion.head(3))
display(df_events.head(3))

,frame_idx,time,x,y,velocity
0,1,18.3,368.498688,368.475006,NaN
1,4,18.4,376.327393,392.465240,252.352920
2,7,18.5,372.444489,408.628235,166.228557


,trial_idx,block_type,was_omission,was_forced_choice,start_time,mouse_init_time,screen_touch_time,reward_collection_time,chose_large,was_shocked
0,1,1,False,True,78.241,78.842,81.672,82.344,True,False
1,2,1,False,True,90.344,95.799,98.387,99.676,True,False
2,3,1,False,True,107.676,108.766,109.963,110.787,True,False


#### Trial Alignment

In [27]:
df1 = demarcate_trials(df_motion, df_events)
df1 = demarcate_latency_from_event(df1, df_events, event_time_col="start_time")
df1 = demarcate_latency_from_event(
    df1,
    df_events,
    event_time_col="reward_collection_time",
    t_after_event=0,
    t_before_event=10e10,
    adjust_backward=False,  # when aligning backwards
)

#### Output

In [29]:
trial2 = df1.loc[df1["trial_idx"] == 2]
display(trial2.iloc[:, :8].head(3))

,time,latency_from_reward_collection_time,latency_from_start_time,trial_idx,frame_idx,x,y,velocity
721,90.4,-9.3,0.0,2.0,2164,496.415741,256.438904,233.854012
722,90.5,-9.2,0.1,2.0,2167,476.291626,244.550781,233.732210
723,90.6,-9.1,0.2,2.0,2170,476.417786,236.670853,78.809384


#### Properties of output

In [30]:
trial2_time = trial2["time"]
tmin, tmax = trial2_time.min().round(2), trial2_time.max().round(2)
print(f"Trial 2 time range: {tmin} to {tmax}")

display(
    df_events.loc[df_events["trial_idx"] == 2,][
        ["trial_idx", "start_time", "reward_collection_time"]
    ]
)

Trial 2 time range: 90.4 to 99.6


,trial_idx,start_time,reward_collection_time
1,2,90.344,99.676
